In [1]:
# set the python path to the parent directory
# this allows us to import from the psychonaut package
import sys
from pathlib import Path
from pprint import pprint

sys.path.append(str(Path.cwd().parent))

In [2]:
retain = []
def err_callback(i, msg):
    retain.append((i, msg))

In [3]:
from psychonaut.firehose.io import length_prefixed_iter
from psychonaut.firehose.simple_reader import iter_events
from collections import defaultdict
import random

sample_segment = Path("../../stream_length_prefixed/stream.1682780863.length-prefixed")
msgs = length_prefixed_iter(sample_segment)
op_codes = defaultdict(int)
sample = []
for i, event in enumerate(iter_events(msgs, err_callback)):
    op_codes[(event[0]['t'], event[0]['op'])] += 1
    if random.random() < 0.5:
        sample.append(event)

op_codes, len(sample)

(defaultdict(int, {('#commit', 1): 29005}), 14445)

In [4]:
len(retain)

1

In [1]:
from carbox.car import read_car


error_target = retain[-1]
read_car(error_target[1])

NameError: name 'retain' is not defined

In [ ]:
len(error_target[1])

117

In [ ]:
jjsample[0]

({'t': '#commit', 'op': 1},
 {'ops': [{'cid': CID('base58btc', 1, 'dag-cbor', '1220427c6e8599870c68566ef338994b153ce5235ff6f7c6384f75e15bd29a6f46a2'),
    'path': 'app.bsky.feed.like/3jujfu2b2lt2g',
    'action': 'create'}],
  'seq': 8982051,
  'prev': CID('base58btc', 1, 'dag-cbor', '1220ea8071132b4749c9370cc985329f9bd13cf74d499b570ce07bb308f5e4e55f8f'),
  'repo': 'did:plc:tnvlrgeen4b5y4uzmvnrljb5',
  'time': '2023-04-29T15:07:50.397Z',
  'blobs': [],
  'blocks': b':\xa2eroots\x81\xd8*X%\x00\x01q\x12 \xecd\xf0\xc1\x84*\xf69\xfdL\xe2p 7\x08U(\xc9\x04\x16\x9c\xd9>f\xf0\x9f\xc4bj\xb0\x7f\xe1gversion\x01\xf8\x01\x01q\x12 B|n\x85\x99\x87\x0chVn\xf38\x99K\x15<\xe5#_\xf6\xf7\xc68Ou\xe1[\xd2\x9aoF\xa2\xa3e$typerapp.bsky.feed.likegsubject\xa2ccidx;bafyreigwjojfw3fthyrzxagnjm5wm2jn2f2ozcmwh3r2k65h4glg7fvqwacurixFat://did:plc:oky5czdrnfjpqslsw2a5iclo/app.bsky.feed.post/3juhxymund22ricreatedAtx\x182023-04-29T15:07:50.211Z\xf4\x04\x01q\x12 \xfa\x9f\x19\x96\xd8~\x94\x12\xd8\xb6\xa7^D\xa2\x06\xa3 \x

In [7]:
from typing import Optional
from pydantic import BaseModel
from datetime import datetime
from dateutil.parser import parse as parse_date


class Like(BaseModel):
    cid: str
    path: str
    action: str
    repo: str
    subject: str
    uri: str
    created_at: datetime

class CreateFollow(BaseModel):
    cid: str
    path: str
    action: str
    repo: str
    subject: str
    created_at: datetime

class Post(BaseModel):
    cid: Optional[str]
    path: str
    action: str
    repo: str
    created_at: datetime
    text: str


likes = []
follows = []
posts = []
errors = []

def dumb_iterator(items):
    for kind, obj, roots, blocks in items:
        if kind.get('t') != '#commit' or kind.get('op') != 1:
            continue

        ops = obj.get('ops')
        if not ops:
            continue

        try:
            if 'app.bsky.feed.like' in ops[0]['path']:
                like = Like(
                    cid=str(ops[0]['cid']),
                    path=ops[0]['path'],
                    action=ops[0]['action'],
                    repo=obj['repo'],
                    created_at=parse_date(obj['time'])
                )
                likes.append(like)
            elif 'app.bsky.graph.follow' in ops[0]['path']:
                follow = CreateFollow(
                    cid=str(ops[0]['cid']),
                    path=ops[0]['path'],
                    action=ops[0]['action'],
                    repo=obj['repo'],
                    subject=blocks[0].decoded['subject'],
                    created_at=parse_date(obj['time'])
                )
                follows.append(follow)
            elif 'app.bsky.feed.post' in ops[0]['path']:
                print(blocks[0].decoded)
            else:
                print(blocks[0].decoded)
                pass

            yield kind, obj, roots, blocks
        except Exception as e:
            errors.append((e, kind, obj, roots, blocks))
            print(e)
            pass


for lmao in dumb_iterator(sample):
    pasiis

len(likes), len(follows)

2 validation errors for Like
subject
  field required (type=value_error.missing)
uri
  field required (type=value_error.missing)


NameError: name 'pasiis' is not defined

In [8]:
from more_itertools import take


def apply_to_commits_with_ops(items, callbacks, error_callback):
    for kind, obj, roots, blocks in items:
        if kind.get('t') != '#commit':
            continue

        ops = obj.get('ops')
        if not ops:
            continue

        for callback in callbacks:
            try:
                callback(ops, obj, roots, blocks)
            except KeyboardInterrupt:
                raise
            except Exception as e:
                error_callback(e, kind, obj, roots, blocks)


errors = []
def collect_errors(e, kind, obj, roots, blocks):
    errors.append((e, kind, obj, roots, blocks))
        

likes = []
target = []
def collect_likes(ops, obj, roots, blocks):
    # {'$type': 'app.bsky.feed.like',
    #  'subject': {'cid': 'bafyreib3yuuxnqh37i7uwekxdtymvqtuerdwy7jfzboyhogrfeupdvakyi',
    # 'uri': 'at://did:plc:2q56lgxb4ccwhic5fv5ew7t4/app.bsky.feed.post/3jujhsymrju2g'},
    # 'createdAt': '2023-04-29T15:44:36.008Z'}
    if 'app.bsky.feed.like' in ops[0]['path']:
        b1 = blocks[0].decoded
        like = Like(
            cid=str(ops[0]['cid']),
            path=ops[0]['path'],
            action=ops[0]['action'],
            repo=obj['repo'],
            subject=b1['subject']['cid'],
            uri=b1['subject']['uri'],
            created_at=parse_date(obj['time'])
        )
        likes.append(like)

        if "zdpuAsY4irZjcHKxiCTLSJXtbVM1cqFVcCtkHxiZTfTL6LCYp" == like.cid:
            target.append((ops, obj, roots, blocks))


posts = []
def collect_posts(ops, obj, roots, blocks):
    if 'app.bsky.feed.post' in ops[0]['path']:
        post = Post(
            cid=str(ops[0]['cid']),
            path=ops[0]['path'],
            action=ops[0]['action'],
            repo=obj['repo'],
            created_at=parse_date(obj['time']),
            text=blocks[0].decoded['text']
        )
        posts.append(post)

class DeleteFollow(BaseModel):
    #cid: str
    path: str
    action: str
    repo: str
    #subject: str
    created_at: datetime

follows = []
def collect_follows(ops, obj, roots, blocks):
    if 'app.bsky.graph.follow' in ops[0]['path']:
        action = ops[0]['action']
        if action == 'create':
            follow = CreateFollow(
                cid=str(ops[0]['cid']),
                path=ops[0]['path'],
                action=action,
                repo=obj['repo'],
                subject=blocks[0].decoded['subject'],
                created_at=parse_date(obj['time'])
            )
        else:
            follow = DeleteFollow(
                path=ops[0]['path'],
                action=action,
                repo=obj['repo'],
                created_at=parse_date(obj['time'])
            )
        follows.append(follow)

unknowns = []
known_paths = {
    'app.bsky.feed.like',
    'app.bsky.graph.follow',
    'app.bsky.feed.post',
}
def collect_unknowns(ops, obj, roots, blocks):
    path = ops[0]['path']
    for known_path in known_paths:
        if known_path in path:
            return
    unknowns.append((path, obj, roots, blocks))



apply_to_commits_with_ops(
    take(100000, sample), 
    [collect_likes, collect_follows, collect_posts], 
    collect_errors
)

len(errors), len(likes), len(follows), len(posts), len(unknowns), len(target)

(61, 6561, 5038, 2127, 0, 1)

In [68]:
errors[0]

(KeyError('subject'),
 {'t': '#commit', 'op': 1},
 {'ops': [{'cid': None,
    'path': 'app.bsky.feed.like/3jujfsw24s22r',
    'action': 'delete'}],
  'seq': 8982112,
  'prev': CID('base58btc', 1, 'dag-cbor', '1220968d62b76f538eb122230ae76e0ff7e91cb99580d8533038fd1477b808aff048'),
  'repo': 'did:plc:jrj3rpch53vkqok4fwvctyno',
  'time': '2023-04-29T15:07:54.882Z',
  'blobs': [],
  'blocks': b':\xa2eroots\x81\xd8*X%\x00\x01q\x12 \xa4\x08A\xe8_\x97\x9f\x12^\x89\xd7.\xf8\xc6C\xdc\xa6\xf6 c-\xa6\xb6\x7f\x02,"Ie^O\x10gversion\x01+\x01q\x12 \x9d\xfe\xfea\xddv\xea=\xca\xe5\x028\x80\xb0\x83y\xd5z\xdf H-o\xdb\xe2u\x92\x89\xf6Gg{\xa2ae\x80al\xf6\xf6\x01\x01q\x12 \xa4\x08A\xe8_\x97\x9f\x12^\x89\xd7.\xf8\xc6C\xdc\xa6\xf6 c-\xa6\xb6\x7f\x02,"Ie^O\x10\xa5cdidx did:plc:jrj3rpch53vkqok4fwvctynocsigX@\xec\x94\xb0>\xdd4\xb3^\xf1:\xfe\xf5\x91y\x89\xeeq\xdfP\xc1j\x81]\t\x8c\x16B\xeb\xd4\xa2\xe2\xef\x1b2\xe5[\x02]\x96\xe2\xf8\x83\xe0\xf8L\x96\xe8\x89p\x9a%\x85\xedc`\x81`Tgfx\x89\xa8lddata\xd8*X%\x00\x01q\x12

In [69]:
likes

[Like(cid='zdpuAsMTXc9HNSd35WVAQ5fwUsiwFK5DSWsMp5gHuSVLFdrqR', path='app.bsky.feed.like/3jujftulhz32o', action='create', repo='did:plc:n227lcznnmabmrdwort4772l', subject='bafyreiaf223yjy4hheczkhkcuiuri72kqmudhk6o5gapg4nslnaadxynly', uri='at://did:plc:oqbijttmayqui2xv3e2xwcdz/app.bsky.feed.post/3jujfhgp2kd2g', created_at=datetime.datetime(2023, 4, 29, 15, 7, 44, 437000, tzinfo=tzutc())),
 Like(cid='zdpuAsupCpAYFmfGmUa88NfRzKvdpQJV8tCtHfeVBQC5sCRtv', path='app.bsky.feed.like/3jujfturomm23', action='create', repo='did:plc:4ggm5sutzou4kkolwqnhbj2q', subject='bafyreifhm5c4mzopqp2osw5d6c2ngjo3pfbgojyso5wa7ffgtpmmtxl76a', uri='at://did:plc:qnd2lqn52ernybdfonq4mc43/app.bsky.feed.post/3jujdmwymi32g', created_at=datetime.datetime(2023, 4, 29, 15, 7, 44, 676000, tzinfo=tzutc())),
 Like(cid='zdpuAxToKnE74JdvJ7nqUuZf7xCtiXFZRJREYzR9EQt3xN9bh', path='app.bsky.feed.like/3jujftwe6w624', action='create', repo='did:plc:4yitncn3mh6urwxzgshgvmnu', subject='bafyreidmtkb4srze572yozbrvveq2ddcgztrfhof5xsupagz

In [72]:
from collections import Counter

Counter([l.uri for l in likes]).most_common(10)

[('at://did:plc:p7gxyfr5vii5ntpwo7f6dhe2/app.bsky.feed.post/3jujgu247h42j',
  179),
 ('at://did:plc:unkqxy3fxjckrxrqstopn3qb/app.bsky.feed.post/3jujfyqd5ll2g',
  95),
 ('at://did:plc:qzuw62ra5pgoy7ohoaj4gd7v/app.bsky.feed.post/3jujglls4pt2o',
  41),
 ('at://did:plc:nvcq6r3nctivcg2rf3vsf4gp/app.bsky.feed.post/3jujd2q6ttl23',
  35),
 ('at://did:plc:fcl2kovn6kqhvtjcavcl4ssh/app.bsky.feed.post/3jujfibltge2j',
  32),
 ('at://did:plc:oqbijttmayqui2xv3e2xwcdz/app.bsky.feed.post/3jujfhgp2kd2g',
  30),
 ('at://did:plc:pdac24jyypwultrppa457voe/app.bsky.feed.post/3jujh2ammth2z',
  29),
 ('at://did:plc:ybugjapko2qap6aofsfelvby/app.bsky.feed.post/3jujhkwnglt2o',
  29),
 ('at://did:plc:2obtldrf6h4jyg4ttea63nbd/app.bsky.feed.post/3jujfzwolid2h',
  27),
 ('at://did:plc:uewxgchsjy4kmtu7dcxa77us/app.bsky.feed.post/3jujgiwgjr22r',
  27)]

In [38]:
follows

[CreateFollow(cid='zdpuAwNGcz7DM9zEuYyfxE1n3XV6eeu6psxGrMdq3oLBBX3Dn', path='app.bsky.graph.follow/3jujfucww6x2z', action='create', repo='did:plc:bclb46tmehgamgtdwfl5d2dx', subject='did:plc:oky5czdrnfjpqslsw2a5iclo', created_at=datetime.datetime(2023, 4, 29, 15, 7, 59, 492000, tzinfo=tzutc())),
 CreateFollow(cid='zdpuB333PLBRqAxDMHo7XeEhGsDKBSZCrUQTqstZnAZ7t1E9X', path='app.bsky.graph.follow/3jujfuijrtd2g', action='create', repo='did:plc:x2wp53pfb7slfsxbxem4rh7v', subject='did:plc:ww4okrwi5pe7qxdrqsphj3o5', created_at=datetime.datetime(2023, 4, 29, 15, 8, 5, 360000, tzinfo=tzutc())),
 CreateFollow(cid='zdpuB2wyx1gvnJkfAv9V6prtEDknEDE9wCjWvxesmyXBnAwe3', path='app.bsky.graph.follow/3jujfva5w3c2p', action='create', repo='did:plc:7aw4rtood3w3yovt4fspc63t', subject='did:plc:ts4ryn3v3drnuzw5xjpeg5f7', created_at=datetime.datetime(2023, 4, 29, 15, 8, 30, 151000, tzinfo=tzutc())),
 CreateFollow(cid='zdpuB26kV5adfPATVxiYha4f7XUXx5UNB78rEg5AvAmq3Gujr', path='app.bsky.graph.follow/3jujfvbju442s'

In [39]:
likes

[Like(cid='zdpuAoBPAaCbK5ZEh9r3JvHRUHr5hAQadrr5MHDJTonWpSSYq', path='app.bsky.feed.like/3jujfvggg3m2j', action='create', repo='did:plc:vqgpg777mglf4zduiolmhlgw', created_at=datetime.datetime(2023, 4, 29, 15, 8, 36, 726000, tzinfo=tzutc())),
 Like(cid='zdpuAvZ8ZL3mEJA6v5LWeVWqp3VhGFVyEp2juTuVUM76ezcXy', path='app.bsky.feed.like/3jujfvpc2ze2s', action='create', repo='did:plc:ghsksroblmcsa2wbtxn5lagk', created_at=datetime.datetime(2023, 4, 29, 15, 8, 46, 12000, tzinfo=tzutc())),
 Like(cid='zdpuAu3227o5LYJ9TwHZQNQ5Att527AZRi2cvZUVRhPu7tH1m', path='app.bsky.feed.like/3jujfw2genl2g', action='create', repo='did:plc:ghsksroblmcsa2wbtxn5lagk', created_at=datetime.datetime(2023, 4, 29, 15, 8, 57, 672000, tzinfo=tzutc())),
 Like(cid='zdpuAm3KPxb3Vr9bcNMwqruZG99a44VqyjhJHhyzmBpYPFnfF', path='app.bsky.feed.like/3jujfwyd2ne2s', action='create', repo='did:plc:rcmdiuszsd2et6t6jfsdstjq', created_at=datetime.datetime(2023, 4, 29, 15, 9, 29, 53000, tzinfo=tzutc())),
 Like(cid='zdpuAnZeRQL3rxaZdrBqkiQVbww

In [ ]:
# Figure out likes

repo <liked> <cid> <time>

In [ ]:
PYTHON_CREATE_SQL ="""
CREATE TABLE IF NOT EXISTS likes (

"""

In [60]:
target[0][-1][0].decoded

{'$type': 'app.bsky.feed.like',
 'subject': {'cid': 'bafyreib3yuuxnqh37i7uwekxdtymvqtuerdwy7jfzboyhogrfeupdvakyi',
  'uri': 'at://did:plc:2q56lgxb4ccwhic5fv5ew7t4/app.bsky.feed.post/3jujhsymrju2g'},
 'createdAt': '2023-04-29T15:44:36.008Z'}

In [46]:
from psychonaut.client import get_ipynb_client_session
from base64 import b64decode

sess = await get_ipynb_client_session()

In [49]:
from psychonaut.api.lexicons.com.atproto.repo.describe_repo import DescribeRepoReq

repo = "did:plc:2jas2nlgpaygmujrjamoftj7"
resolved_handle = await DescribeRepoReq(repo=repo).do_xrpc(sess)
resolved_handle

DescribeRepoResp(handle='pleasebegneiss.bsky.social', did='did:plc:2jas2nlgpaygmujrjamoftj7', didDoc={'@context': ['https://www.w3.org/ns/did/v1', 'https://w3id.org/security/suites/secp256k1-2019/v1'], 'id': 'did:plc:2jas2nlgpaygmujrjamoftj7', 'alsoKnownAs': ['at://pleasebegneiss.bsky.social'], 'verificationMethod': [{'id': '#atproto', 'type': 'EcdsaSecp256k1VerificationKey2019', 'controller': 'did:plc:2jas2nlgpaygmujrjamoftj7', 'publicKeyMultibase': 'zQYEBzXeuTM9UR3rfvNag6L3RNAs5pQZyYPsomTsgQhsxLdEgCrPTLgFna8yqCnxPpNT7DBk6Ym3dgPKNu86vt9GR'}], 'service': [{'id': '#atproto_pds', 'type': 'AtprotoPersonalDataServer', 'serviceEndpoint': 'https://bsky.social'}]}, collections=['app.bsky.actor.profile', 'app.bsky.feed.like', 'app.bsky.feed.post', 'app.bsky.feed.repost', 'app.bsky.graph.follow'], handleIsCorrect=True)

In [50]:
from psychonaut.api.lexicons.com.atproto.repo.get_record import GetRecordReq


(await GetRecordReq(
    repo=repo,
    collection="app.bsky.feed.like",
    rkey="3jujhvs4ysm2g",
).do_xrpc(sess)).dict()

{'uri': 'at://did:plc:2jas2nlgpaygmujrjamoftj7/app.bsky.feed.like/3jujhvs4ysm2g',
 'cid': 'bafyreidjwrctbtonw4oy6oitdjs3hmw7we57xogq5yh2eaemoblx7bkmne',
 'value': {'$type': 'app.bsky.feed.like',
  'subject': {'cid': 'bafyreib3yuuxnqh37i7uwekxdtymvqtuerdwy7jfzboyhogrfeupdvakyi',
   'uri': 'at://did:plc:2q56lgxb4ccwhic5fv5ew7t4/app.bsky.feed.post/3jujhsymrju2g'},
  'createdAt': '2023-04-29T15:44:36.008Z'}}

In [73]:
'at://did:plc:p7gxyfr5vii5ntpwo7f6dhe2/app.bsky.feed.post/3jujgu247h42j'

(await GetRecordReq(
    repo="did:plc:p7gxyfr5vii5ntpwo7f6dhe2",
    collection="app.bsky.feed.post",
    rkey="3jujgu247h42j",
).do_xrpc(sess)).dict()

{'uri': 'at://did:plc:p7gxyfr5vii5ntpwo7f6dhe2/app.bsky.feed.post/3jujgu247h42j',
 'cid': 'bafyreibqijait2pldsmzmrit4cavjl4fbfbxn5pqg22aylbtxm2jofjkpm',
 'value': {'text': 'I think ostracizing fascists is good, actually. They used to wear hoods for a reason',
  '$type': 'app.bsky.feed.post',
  'createdAt': '2023-04-29T15:25:43.918Z'}}

In [55]:
from psychonaut.api.lexicons.com.atproto.sync.get_blob import GetBlobReq

(await GetBlobReq(
    did=repo,
    cid="zdpuAsY4irZjcHKxiCTLSJXtbVM1cqFVcCtkHxiZTfTL6LCYp"
).do_xrpc(sess)).dict()

ClientResponseError: 400, message='Bad Request', url=URL('https://bsky.social/xrpc/com.atproto.sync.getBlob?did=did:plc:2jas2nlgpaygmujrjamoftj7&cid=zdpuAsY4irZjcHKxiCTLSJXtbVM1cqFVcCtkHxiZTfTL6LCYp')